In [1]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
from REINFORCE_soft import MultiStockEnv, Agent, play_one_episode, maybe_make_dir
import market_tickers
from model_config import model_parameters
import tensorflow as tf

Wall time: 2.27 s


In [1]:
#------ small test-sweep ------#
#GAMMA_TRADES = [0.1, 1, 10, 60, 100]
#GAMMA_RISKS = [0.1, 1, 18, 56, 100, 562, 1000]
#------ extended-boyd-sweep ------#
GAMMA_TRADES =  [0.1, 0.5, 1, 2, 3, 4, 5, 5.5, 6, 6.5, 7, 7.5, 8, 9, 10, 11, 12, 15, 20, 30, 45, 60, 100, 200]
GAMMA_RISKS = [0.1, 0.178, 0.316, 0.562, 1, 2, 3, 6, 10, 18, 32, 56, 100, 178, 316, 562, 1000, 2000, 5000, 10000, 20000]
#---------------------------------#

GAMMA_PAIRS = []
for tr in GAMMA_TRADES:
    for ri in GAMMA_RISKS:
        GAMMA_PAIRS.append((tr, ri))
        
#format: (gamma_trade, gamma_risk)
print('number of pairs: ', len(GAMMA_PAIRS))
#GAMMA_PAIRS

number of pairs:  504


In [3]:
# papermill parameters
SEED_LIST = [
             0,
             #1111,
             #2222, 
             #3333, 
             #4444, 
             #5555,
             #6666,
             #7777,
             #8888,
             #9999
            ]

#TICKERS = market_tickers.DOW_30_TICKER
TICKERS = ['AAPL', 'AMD', 'CSCO', 'F', 'INTC']
MARKET_NAME = 'TEST_5'
MODEL_BASE_NAME = 'RL_CNN' # use either | 'RL_CNN' | 'RL_str_fcast' | 'RL_all_inp' |
FROM = '2017-01-03' # start of training set
UNTIL = '2019-01-01' # end of training set
NB_EPISODES = 200 # number of episodes to train for
SAVE_EVERY = 200 # (used 100 when not from tretrained) save model weights every time this amount of episodes pass

In [4]:
# other constants and parameters
TICKERS.sort()
FILE_PERIOD = '1d' # weekly='5d', daily='1d'
DAYS_IN_EPISODE = 30 # 365 for one-year long episodes (conditions checked at end of episode)
EPISODE_DRAW_DISTRIBUTION = 'uniform' # 'uniform' or 'geometric'. select starting point of eposide according to this distribution when generated
HALF_SPREAD = 0.0005/2.0 # 'a' in transaction cost function
NONLIN_COEFF = 1.0 # 'b' transaction cost function
POWER = 1.5 # power for change in poertfolio vector used in transaction cost
#GAMMA_RISK, GAMMA_TRADE, GAMMA_HOLD = 18, 6.5, 0.0 # relative importance of risk, trading cost, and holding cost
INIT_PORTFOLIO = 100000000.0 # initial portfolio value
#model_name = f'REINFORCE_CNN' #_seeded_{UNTIL} # give model a name to distinguish saved files
#NB_EPISODES = 300 #2000
MODE = 'train' # train or test mode
#FROM_PRETRAINED = False
DATA_DIR = f'../../data/{MARKET_NAME}/preprocessed_data/'

# Tune and double-check these before every test:
USE_FORECASTS = model_parameters[MODEL_BASE_NAME]['use_forecasts'] # whether to use forecasts as inputs to agent policy net or not
NB_FORECASTS = model_parameters[MODEL_BASE_NAME]['nb_forecasts'] # number of forecasts to use in state. Set to None for no forecasts
FORECAST_TYPE = model_parameters[MODEL_BASE_NAME]['forecast_type'] # use 'strong' or 'weak' forecasts (if no forecasts given, this does not matter)
USE_CNN_STATE = model_parameters[MODEL_BASE_NAME]['use_cnn_state'] # whether to pass log-rets block for CNN part of policy net (set to false for forecast only)

In [5]:
%%time
##########################
### main training loop ###
##########################

# define investor preferences to train for
#------ extended-boyd-sweep ------#
#GAMMA_TRADES = [5.5, 6, 6.5, 7, 7.5, 8, 9, 10, 11, 12, 15, 20, 30]
#GAMMA_TRADES = [4, 5]
#GAMMA_TRADES = [0.1, 0.5, 1, 2, 3, 45, 60, 100, 200]
#GAMMA_RISKS = [0.1, 0.178, 0.316, 0.562, 1, 2, 3, 6, 10, 18, 32, 56, 100, 178, 316, 562, 1000, 2000, 5000, 10000, 20000]
#---------------------------------#
counter = 0

for RANDOM_SEED in SEED_LIST:
    model_name = f'{MODEL_BASE_NAME}_seed_{RANDOM_SEED}'
    maybe_make_dir(f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/seed_{RANDOM_SEED}')
    
    for (GAMMA_TRADE, GAMMA_RISK) in GAMMA_PAIRS:
        # set random generator seed for tf and np
        # this is probably unnecessary because it is done when env is initialised but just incase
        tf.random.set_seed(RANDOM_SEED)
        np.random.seed(RANDOM_SEED)
        counter += 1

        # check if test lready done. skip if so
        #if not FROM_PRETRAINED:
            #skip = os.path.exists(f'experiments/12_assets/00_seeded/seed_{RANDOM_SEED}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}') # check if already exists
        #else:
        
        skip = os.path.exists(f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/seed_{RANDOM_SEED}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}')

        if not skip: # if not exists, do test

            print('#############################################################################')
            print(f'##\t[{counter}/{len(GAMMA_PAIRS)}]\tmodel={MODEL_BASE_NAME},\tSEED={RANDOM_SEED},\ttrade={GAMMA_TRADE},\trisk={GAMMA_RISK}\t##')
            print('#############################################################################')        

            # create agent and environment
            env = MultiStockEnv(tickers=TICKERS, 
                                from_date=FROM,
                                until=UNTIL, 
                                #nb_episodes=100,
                                cash_key='USDOLLAR', 
                                gamma_risk=GAMMA_RISK, 
                                gamma_trade=GAMMA_TRADE,
                                half_spread=HALF_SPREAD, 
                                nonlin_coef=NONLIN_COEFF, 
                                power=POWER, 
                                datadir=DATA_DIR, 
                                state_lookback_window=20, 
                                distribution=EPISODE_DRAW_DISTRIBUTION,
                                days_duration=DAYS_IN_EPISODE, 
                                mode='train', 
                                random_seed=RANDOM_SEED,
                                init_portfolio=INIT_PORTFOLIO, 
                                period_in_file_name=FILE_PERIOD, 
                                nb_forecasts=NB_FORECASTS, 
                                forecast_type=FORECAST_TYPE,
                                use_CNN_state=USE_CNN_STATE,
                                verbose=False)

            agent = Agent(alpha=0.001, 
                          gamma=0.99, 
                          n_assets=len(TICKERS)+1, 
                          tau=5, 
                          lookback_window=20, 
                          n_feature_maps=len(TICKERS)+1,
                          use_forecasts=USE_FORECASTS,
                          use_CNN_state=USE_CNN_STATE)

            # initialise policy by one forward pass then load pretrained weights
            #if FROM_PRETRAINED: 
                #agent.choose_action(env._get_obs())
                #agent.load(pretrined_weights_dir)

            # configure folders
            experiments_folder = f'experiments/{MARKET_NAME}'
            from_seeded_folder = f'{experiments_folder}/seeded'
            base_model_folder = f'{from_seeded_folder}/{MODEL_BASE_NAME}'
            seed_folder = f'{base_model_folder}/seed_{RANDOM_SEED}'
            #if FROM_PRETRAINED:
            #    pretrained_folder = f'{nb_assets_folder}/0_further_trained'
            #    preference_folder = f'{pretrained_folder}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}'
            #else:
            preference_folder = f'{seed_folder}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}'
            models_folder = f'{preference_folder}/models' # where saved models will be saved
            losses_folder = f'{preference_folder}/losses' # where losses will be saved for plotting training progress
            fig_folder = f'{preference_folder}/figures'

            # create dirictories for saving outputs to (if they don't exist already)
            #maybe_make_dir(nb_assets_folder)
            maybe_make_dir(seed_folder)
            maybe_make_dir(preference_folder)
            maybe_make_dir(models_folder)
            maybe_make_dir(losses_folder)
            maybe_make_dir(fig_folder)

            if MODE == 'train':
                # play the game num_episodes times and update weights according to loss
                losses = []
                for e in range(NB_EPISODES):

                    # save model weights every few episodes
                    if (e%SAVE_EVERY == 0) and (e!=0):
                        agent.save(f'{models_folder}/{model_name}_{e}')
                        # save losses of training period
                        np.save(f'{losses_folder}/{model_name}_{MODE}_{e}_losses.npy', np.array(losses))

                    t0 = datetime.datetime.now()
                    try:
                        loss = play_one_episode(agent, env, MODE)
                    except Exception as err:
                        print(f'\t*** error in episode {e}:', err)
                        loss = np.nan#5.0 # just an arbitraty number that seems high compared to usual losses
                    dt = datetime.datetime.now() - t0
                    losses.append(loss) # append episode loss
                    # print progress
                    if (e%10 == 0):
                        print(f"episode: {e + 1}/{NB_EPISODES}, loss: {loss:.4f}, duration: {dt}")


                # save the weights and losses when done	
                agent.save(f'{models_folder}/{model_name}_{NB_EPISODES}')
                np.save(f'{losses_folder}/{model_name}_{MODE}_{NB_EPISODES}_losses.npy', np.array(losses))

            # plot losses
            save_fig = True

            # load losses file
            file = f'{losses_folder}/{model_name}_{MODE}_{NB_EPISODES}_losses.npy'
            loaded_losses = np.load(file)

            # plot losses
            plt.figure(figsize=(14,5))
            plt.plot(loaded_losses)
            plt.title(f'{model_name} Training Loss')
            plt.xlabel('Episode Number')
            plt.ylabel(r'Loss: $-G \approx -\mathbb{E}\left[ \sum_{k=0}^{T} \gamma^k R_t \right]$')
            #plt.yscale('symlog')
            if save_fig:
                plt.savefig(f'{fig_folder}/{model_name}_losses.png', dpi=150, facecolor=None, edgecolor=None, bbox_inches='tight')
            plt.clf()

        else:
            print(f'\talready exists: trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}')
            print('\tskip to next.')

	already exists: trade_0.1_risk_0.1
	skip to next.
	already exists: trade_0.1_risk_1
	skip to next.
	already exists: trade_0.1_risk_18
	skip to next.
	already exists: trade_0.1_risk_56
	skip to next.
	already exists: trade_0.1_risk_100
	skip to next.
	already exists: trade_0.1_risk_562
	skip to next.
	already exists: trade_0.1_risk_1000
	skip to next.
#############################################################################
##	[8/35]	model=RL_CNN,	SEED=0,	trade=1,	risk=0.1	##
#############################################################################
episode: 1/200, loss: 0.0060, duration: 0:00:02.159071
episode: 11/200, loss: -0.0262, duration: 0:00:00.496245
episode: 21/200, loss: 0.0293, duration: 0:00:00.522427
episode: 31/200, loss: -0.0550, duration: 0:00:00.467136
episode: 41/200, loss: 0.0093, duration: 0:00:00.471478
episode: 51/200, loss: -0.0511, duration: 0:00:00.464695
episode: 61/200, loss: -0.0106, duration: 0:00:00.469389
episode: 71/200, loss: 0.0078, duration: 0

episode: 171/200, loss: 0.0074, duration: 0:00:00.498134
episode: 181/200, loss: -0.0034, duration: 0:00:00.503130
episode: 191/200, loss: 0.0030, duration: 0:00:00.500675
#############################################################################
##	[14/35]	model=RL_CNN,	SEED=0,	trade=1,	risk=1000	##
#############################################################################
episode: 1/200, loss: 1.9436, duration: 0:00:00.518962
episode: 11/200, loss: 0.4899, duration: 0:00:00.499999
episode: 21/200, loss: 1.9533, duration: 0:00:00.562999
episode: 31/200, loss: 1.9280, duration: 0:00:00.500001
episode: 41/200, loss: 0.8911, duration: 0:00:00.502559
episode: 51/200, loss: 0.4087, duration: 0:00:00.510096
episode: 61/200, loss: 0.3752, duration: 0:00:00.500001
episode: 71/200, loss: 0.3295, duration: 0:00:00.515625
episode: 81/200, loss: 0.2788, duration: 0:00:00.519121
episode: 91/200, loss: 0.2120, duration: 0:00:00.502730
episode: 101/200, loss: 0.0848, duration: 0:00:00.503040
e

#############################################################################
##	[20/35]	model=RL_CNN,	SEED=0,	trade=10,	risk=562	##
#############################################################################
episode: 1/200, loss: 1.1040, duration: 0:00:00.518720
episode: 11/200, loss: 0.2679, duration: 0:00:00.494722
episode: 21/200, loss: 1.1270, duration: 0:00:00.515625
episode: 31/200, loss: 1.0815, duration: 0:00:00.513987
episode: 41/200, loss: 0.5106, duration: 0:00:00.515626
episode: 51/200, loss: 0.2177, duration: 0:00:00.512006
episode: 61/200, loss: 0.2076, duration: 0:00:00.515625
episode: 71/200, loss: 0.1893, duration: 0:00:00.500002
episode: 81/200, loss: 0.1673, duration: 0:00:00.511103
episode: 91/200, loss: 0.1228, duration: 0:00:00.499998
episode: 101/200, loss: 0.0496, duration: 0:00:00.500613
episode: 111/200, loss: 0.0088, duration: 0:00:00.515623
episode: 121/200, loss: 0.0076, duration: 0:00:00.499999
episode: 131/200, loss: 0.0093, duration: 0:00:00.516245
ep

episode: 1/200, loss: 0.2597, duration: 0:00:00.519663
episode: 11/200, loss: 0.0514, duration: 0:00:00.527249
episode: 21/200, loss: 0.3231, duration: 0:00:00.515626
episode: 31/200, loss: 0.2549, duration: 0:00:00.516315
episode: 41/200, loss: 0.1397, duration: 0:00:00.515621
episode: 51/200, loss: 0.0412, duration: 0:00:00.500694
episode: 61/200, loss: 0.0505, duration: 0:00:00.515629
episode: 71/200, loss: 0.0647, duration: 0:00:00.515620
episode: 81/200, loss: 0.0984, duration: 0:00:00.502149
episode: 91/200, loss: 0.0486, duration: 0:00:00.516259
episode: 101/200, loss: 0.0264, duration: 0:00:00.515625
episode: 111/200, loss: 0.0020, duration: 0:00:00.515625
episode: 121/200, loss: 0.0023, duration: 0:00:00.516280
episode: 131/200, loss: 0.0103, duration: 0:00:00.515757
episode: 141/200, loss: 0.0046, duration: 0:00:00.512225
episode: 151/200, loss: 0.0080, duration: 0:00:00.516322
episode: 161/200, loss: 0.0091, duration: 0:00:00.516247
episode: 171/200, loss: 0.0050, duration: 

C:\Users\Ruan Pretorius\anaconda3\lib\site-packages\ipykernel_launcher.py:134: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


...

<Figure size 1400x500 with 0 Axes>

<Figure size 1400x500 with 0 Axes>

<Figure size 1400x500 with 0 Axes>

<Figure size 1400x500 with 0 Axes>

<Figure size 1400x500 with 0 Axes>

In [6]:
# double check the end of training
#env.until

'2019-01-01'

In [7]:
print('training done.')

training done.


In [8]:
### Now repeat this for the different RL models.
### remember to update the gamma-pairs, model base name, and model specs/setup in beginning of notebook!